In [1]:
import torch
from torch import nn

In [55]:
X = torch.arange(9).reshape(3,3).type(torch.float32)
print(X)

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])


In [56]:
K = torch.arange(4).reshape(2,2).type(torch.float32)
print(K)

tensor([[0., 1.],
        [2., 3.]])


In [39]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range (Y.shape[0]):
        for j in range (Y.shape[1]):
            Y[i, j] = (X[i : i + h, j : j + w] * K).sum()
    return Y


In [57]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [102]:
X = (torch.arange(16) + 1).type(torch.float32).reshape(1,1,4,4)
print(X)
K0 = torch.Tensor([[0,-1,0], [-1,5,-1], [0,-1,0]]).type(torch.float32).reshape(1,1,3,3)
K1 = torch.Tensor([[-1,0,1], [1,-1,0], [0,1,-1]]).type(torch.float32).reshape(1,1,3,3)
K = torch.cat((K0, K1), dim=0)
print(K)
b0 = torch.Tensor([5.])
b1 = torch.Tensor([0.])
b = torch.cat((b0, b1), dim=0)
print(b)
conv2d = torch.nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, bias=True)
print(conv2d.weight.shape)
print(conv2d.bias.shape)
conv2d.weight = nn.Parameter(K)
conv2d.bias = nn.Parameter(b)
Z = conv2d(X)
print(Z[0][0])
print(Z[0][1])

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])
tensor([[[[ 0., -1.,  0.],
          [-1.,  5., -1.],
          [ 0., -1.,  0.]]],


        [[[-1.,  0.,  1.],
          [ 1., -1.,  0.],
          [ 0.,  1., -1.]]]])
tensor([5., 0.])
torch.Size([2, 1, 3, 3])
torch.Size([2])
tensor([[11., 12.],
        [15., 16.]], grad_fn=<SelectBackward0>)
tensor([[0., 0.],
        [0., 0.]], grad_fn=<SelectBackward0>)


In [53]:
X = torch.ones((6,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
# vertical edges filter, when pass throw one to zero or throw zero to one
K = torch.Tensor([[1, -1]])
print(K.shape)
Y = corr2d(X, K)
print(Y)

torch.Size([1, 2])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [ ]:
Z = corr2d(X.T, K)
print(X.T)
# not found
print(Z)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [9]:
X = X.reshape(1,1,6,8)

In [12]:
Y = Y.reshape(1,1,6,7)

In [13]:
Y

tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])

In [14]:
conv2d = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(1,2))

In [15]:
optimizer = torch.optim.SGD(conv2d.parameters(), lr=0.1)

In [16]:
criterion = nn.MSELoss()

In [17]:
for i in range(200):
    optimizer.zero_grad()
    Y_hat = conv2d(X)
    l = criterion(Y_hat, Y)
    l.backward()
    optimizer.step()
    if (i + 1) % 20 == 0:
        print(f'batch {i + 1}, loss { l.item():.3f} ')

batch 20, loss 0.165 
batch 40, loss 0.050 
batch 60, loss 0.015 
batch 80, loss 0.005 
batch 100, loss 0.001 
batch 120, loss 0.000 
batch 140, loss 0.000 
batch 160, loss 0.000 
batch 180, loss 0.000 
batch 200, loss 0.000 


In [18]:
print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[ 0.9961, -0.9961]]]], requires_grad=True)
Parameter containing:
tensor([-2.2425e-05], requires_grad=True)


In [20]:
conv2d = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
X = torch.randn((1,1,8,8))
conv2d(X).shape

torch.Size([1, 1, 8, 8])

In [21]:
conv2d.weight.shape

torch.Size([1, 1, 3, 3])

In [35]:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
# 1 batch, 3 chanels, 8 x 8
X = torch.randn((1,3,8,8))
Z = conv2d(X)
print(Z.shape)
Z[0][28]

torch.Size([1, 32, 8, 8])


tensor([[-0.1636,  0.0375, -0.3740,  0.1032, -0.6506,  0.7263, -0.2048, -0.0845],
        [-0.1678, -0.6150,  0.4872, -0.3901, -0.6995, -0.7752,  0.7438, -0.4703],
        [-0.3539,  0.5536, -0.7808, -0.3305, -0.1008, -0.1421, -0.3280,  0.0506],
        [ 0.2813, -0.4818,  0.6427, -0.2605, -0.5786,  0.5880, -0.2122, -0.1324],
        [-0.2952, -1.4465, -0.1513,  0.5233,  0.6449, -0.0873, -0.6032,  0.2751],
        [-0.3967, -0.7092, -0.1534,  0.2536, -0.6453,  0.3957,  0.1839, -0.2833],
        [ 0.4214, -0.2917, -0.4039, -0.4224,  0.5376, -0.3361, -0.3489, -0.1443],
        [-0.3132,  0.2131,  0.1033, -0.7570, -0.2428, -0.1499, -0.3422, -0.0019]],
       grad_fn=<SelectBackward0>)

In [ ]:
# out 1 batch, 32 chanels, 3 x 3
conv2d.weight.shape

torch.Size([32, 3, 3, 3])

In [24]:
X = torch.arange(16).reshape(1,1,4,4).type(torch.float)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [27]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [30]:
pool2d = nn.MaxPool2d((2,3), padding=(1,1), stride=(2,3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [31]:
X = torch.cat((X, X+1), dim=1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [32]:
pool2d = nn.MaxPool2d((3,3), padding=(1,1), stride=(2,2))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])